# Assignment - Build your fantasy teams for IND v SL 3 match T20 series & find out your avg points per team

In [4]:
#!pip install pulp

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

### Collecting data of India Vs Srilanka 3 T20i match series 2022

In [6]:
df1 = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')
df2 = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')
df3 = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')

# 1st match

## First T20 match Ind vs SL

In [7]:
df1

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1
5,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1
6,3692,38632,Y Chahal,7910,IND,2,BOWL,40.62,8.5,0,1,35,1
7,3692,38632,L Kumara,10928,SL,5,BOWL,34.56,8.5,0,1,35,1
8,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1
9,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1


In [8]:
df1.dtypes

SeriesId         int64  
MatchId          int64  
Player           object 
PlayerId         int64  
Team             object 
TeamId           int64  
Role             object 
Sel              float64
Credits          float64
Series Points    int64  
PlayingXI        int64  
Points           int64  
DT               int64  
dtype: object

In [9]:
df1 = df1.join(pd.get_dummies(df1[['Role', 'Team']]))
df1['PlayingXI'] = 1

teams = list(df1.Team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df1.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj),'Points'] for obj in df1.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Credits']] for obj in df1.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'PlayingXI']] for obj in df1.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team1}"]] for obj in df1.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team1}"]] for obj in df1.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team2}"]] for obj in df1.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), f"Team_{team2}"]] for obj in df1.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_WK']] for obj in df1.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BAT']] for obj in df1.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_BOWL']] for obj in df1.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df1.loc[(obj), 'Role_AR']] for obj in df1.index) <= 4

mo.solve()

# output as a column in df1
df1['pickup_status'] = 0

for obj in df1.index:
    if pickup_status[obj].varValue:
        df1['pickup_status'][obj] = 1

fdf1 = df1[df1['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [10]:
fdf1

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1,0,1,0,0,1,0,1
1,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1,0,1,0,0,0,1,1
2,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1,0,1,0,0,1,0,1
3,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1,0,0,0,1,1,0,1
4,3692,38632,V Iyer,10917,IND,2,AR,53.22,8.5,0,1,50,1,1,0,0,0,1,0,1
5,3692,38632,R Jadeja,587,IND,2,AR,74.73,9.0,0,1,34,1,1,0,0,0,1,0,1
6,3692,38632,D Shanaka,8422,SL,5,AR,50.75,9.0,0,1,32,1,1,0,0,0,0,1,1
7,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1,0,0,1,0,1,0,1
8,3692,38632,Y Chahal,7910,IND,2,BOWL,40.62,8.5,0,1,35,1,0,0,1,0,1,0,1
9,3692,38632,L Kumara,10928,SL,5,BOWL,34.56,8.5,0,1,35,1,0,0,1,0,0,1,1


In [11]:
sum(fdf1.Credits)

97.0

In [12]:
sum(fdf1.Points)

653

In [13]:
np.mean(fdf1.Points)

59.36363636363637

### Average points for 1st Match is 59.36

# *

# 2nd Match

## Second T20 match Ind vs SL

In [14]:
df2

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1
2,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1
3,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1
4,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1
5,3692,38637,D Gunathilaka,8387,SL,5,BAT,4.45,8.5,0,0,1,56,1
6,3692,38637,L Kumara,10928,SL,5,BOWL,46.78,8.5,35,1,1,52,1
7,3692,38637,Y Chahal,7910,IND,2,BOWL,54.21,8.5,35,1,1,39,1
8,3692,38637,B Kumar,1726,IND,2,BOWL,63.78,8.5,76,1,1,37,1
9,3692,38637,D Chameera,8393,SL,5,BOWL,72.85,9.0,30,1,1,31,1


In [15]:
df2 = df2.join(pd.get_dummies(df2[['Role', 'Team']]))
df2['PlayingXI'] = 1

teams = list(df2.Team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df2.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj),'Points'] for obj in df2.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Credits']] for obj in df2.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'PlayingXI']] for obj in df2.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), f"Team_{team1}"]] for obj in df2.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), f"Team_{team1}"]] for obj in df2.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), f"Team_{team2}"]] for obj in df2.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), f"Team_{team2}"]] for obj in df2.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_WK']] for obj in df2.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_WK']] for obj in df2.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT']] for obj in df2.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BAT']] for obj in df2.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL']] for obj in df2.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_BOWL']] for obj in df2.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR']] for obj in df2.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df2.loc[(obj), 'Role_AR']] for obj in df2.index) <= 4

mo.solve()

# output as a column in df2
df2['pickup_status'] = 0

for obj in df2.index:
    if pickup_status[obj].varValue:
        df2['pickup_status'][obj] = 1

fdf2 = df2[df2['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [16]:
fdf2

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1,0,1,0,0,1,0,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1,0,1,0,0,0,1,1
2,3692,38637,D Gunathilaka,8387,SL,5,BAT,4.45,8.5,0,0,1,56,1,0,1,0,0,0,1,1
3,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1,0,0,0,1,1,0,1
4,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1,1,0,0,0,1,0,1
5,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1,1,0,0,0,0,1,1
6,3692,38637,L Kumara,10928,SL,5,BOWL,46.78,8.5,35,1,1,52,1,0,0,1,0,0,1,1
7,3692,38637,Y Chahal,7910,IND,2,BOWL,54.21,8.5,35,1,1,39,1,0,0,1,0,1,0,1
8,3692,38637,B Kumar,1726,IND,2,BOWL,63.78,8.5,76,1,1,37,1,0,0,1,0,1,0,1
9,3692,38637,D Chameera,8393,SL,5,BOWL,72.85,9.0,30,1,1,31,1,0,0,1,0,0,1,1


In [17]:
sum(fdf2.Credits)

96.0

In [18]:
sum(fdf2.Points)

687

In [19]:
np.mean(fdf2.Points)

62.45454545454545

### Average points for 2nd Match is 62.45

# *

# 3rd match

## Third T20 match Ind vs SL

In [20]:
df3

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points
0,3692,38642,S Iyer,9428,IND,2,BAT,88.80,8.5,196.0,2,1,100
1,3692,38642,R Jadeja,587,IND,2,AR,87.60,9.0,127.0,2,1,31
2,3692,38642,S Samson,8271,IND,2,WK,85.50,8.0,59.0,1,1,35
3,3692,38642,D Shanaka,8422,SL,5,AR,82.30,9.0,109.0,2,1,105
4,3692,38642,P Nissanka,13682,SL,5,BAT,82.00,9.5,102.0,1,1,-1
5,3692,38642,R Sharma,576,IND,2,BAT,81.45,10.5,78.2,1,1,10
6,3692,38642,D Chameera,8393,SL,5,BOWL,65.41,9.0,61.0,2,1,31
7,3692,38642,D Gunathilaka,8387,SL,5,BAT,60.51,8.5,56.0,1,1,2
8,3692,38642,L Kumara,10928,SL,5,BOWL,60.44,8.5,87.0,2,1,60
9,3692,38642,C Asalanka,10934,SL,5,BAT,57.96,8.5,76.0,1,1,9


In [21]:
df3 = df3.join(pd.get_dummies(df3[['Role', 'Team']]))
df3['PlayingXI'] = 1

teams = list(df3.Team.unique())
team1, team2 = teams[0], teams[1]

# define decision variable
pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df3.index), cat = 'Binary' )

# define objective function
mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj),'Points'] for obj in df3.index])

# define constraints
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Credits']] for obj in df3.index) <= 100

# 11 players
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'PlayingXI']] for obj in df3.index) == 11

# team constraints
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), f"Team_{team1}"]] for obj in df3.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), f"Team_{team1}"]] for obj in df3.index) <= 7

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), f"Team_{team2}"]] for obj in df3.index) >= 4
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), f"Team_{team2}"]] for obj in df3.index) <= 7

# role constraints
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_WK']] for obj in df3.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_WK']] for obj in df3.index) <= 4

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BAT']] for obj in df3.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BAT']] for obj in df3.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BOWL']] for obj in df3.index) >= 3
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_BOWL']] for obj in df3.index) <= 6

mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_AR']] for obj in df3.index) >= 1
mo += pulp.lpSum([pickup_status[obj]*df3.loc[(obj), 'Role_AR']] for obj in df3.index) <= 4

mo.solve()

# output as a column in df3
df3['pickup_status'] = 0

for obj in df3.index:
    if pickup_status[obj].varValue:
        df3['pickup_status'][obj] = 1

fdf3 = df3[df3['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)

In [22]:
fdf3

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,Role_AR,Role_BAT,Role_BOWL,Role_WK,Team_IND,Team_SL,pickup_status
0,3692,38642,S Iyer,9428,IND,2,BAT,88.80,8.5,196.0,2,1,100,0,1,0,0,1,0,1
1,3692,38642,R Sharma,576,IND,2,BAT,81.45,10.5,78.2,1,1,10,0,1,0,0,1,0,1
2,3692,38642,C Asalanka,10934,SL,5,BAT,57.96,8.5,76.0,1,1,9,0,1,0,0,0,1,1
3,3692,38642,S Samson,8271,IND,2,WK,85.50,8.0,59.0,1,1,35,0,0,0,1,1,0,1
4,3692,38642,D Chandimal,6245,SL,5,WK,25.80,8.5,30.0,0,1,36,0,0,0,1,0,1,1
5,3692,38642,D Shanaka,8422,SL,5,AR,82.30,9.0,109.0,2,1,105,1,0,0,0,0,1,1
6,3692,38642,C Karunaratne,7952,SL,5,AR,14.50,8.5,31.0,0,1,47,1,0,0,0,0,1,1
7,3692,38642,L Kumara,10928,SL,5,BOWL,60.44,8.5,87.0,2,1,60,0,0,1,0,0,1,1
8,3692,38642,R Bishnoi,14659,IND,2,BOWL,48.93,8.0,0.0,0,1,37,0,0,1,0,1,0,1
9,3692,38642,M Siraj,10808,IND,2,BOWL,42.51,8.5,0.0,0,1,41,0,0,1,0,1,0,1


In [23]:
sum(fdf3.Credits)

94.5

In [24]:
sum(fdf3.Points)

550

In [25]:
np.mean(fdf3.Points)

50.0

### Average points for 3rd Match is 50